# Retrieving match history per player

In [2]:
import time

from itertools import chain

import requests as req
import pandas as pd
import numpy as np

from settings import TOKEN

## Goal

Create a unique collection of matches id from the ladder, in order to retrieve the final match data.

## 1. Get the players account_id list

In [3]:
players = pd.read_pickle("raw_data/players_pool_account.pkl")
players.head()

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,accountId
0,DIAMOND,I,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,90,200,187,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg
1,DIAMOND,I,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,0,101,79,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4
2,DIAMOND,I,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,52,160,146,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0
3,DIAMOND,I,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,3,264,261,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY
4,DIAMOND,I,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,30,448,434,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg


In [4]:
player_id_list = players["accountId"].to_list()

## 2. Check the API method

- needs accountId
- needs queue id (420 for 5v5 Ranked Solo Games)
- needs begin time **1593561600 = first of july**

In [5]:
def get_matchlist(account_id: str, token: str, begin_time = 1593561600, queue_id = 420):
    begin_index = 0
    base_url = f"https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account"
    url= f"{base_url}/{account_id}?queue={queue_id}&api_key={token}&beginTime={begin_time}&beginIndex={begin_index}"
    res = req.get(url)
    
    return res.json()

In [6]:
match_list = get_matchlist(player_id_list[0], TOKEN)

In [11]:
match_list["matches"][0]

{'platformId': 'EUW1',
 'gameId': 4671787510,
 'champion': 64,
 'queue': 420,
 'season': 13,
 'timestamp': 1592764299945,
 'role': 'NONE',
 'lane': 'JUNGLE'}

## 3. Adjust the Function to make multiple calls

- Flattens the list of matches into one whole list, easier to work with using pandas.
- Gets all the matchId records which is the value we care about
- Saves to a pickle file

In [47]:
def get_match_history(players_list: list, token: str, start=0 , end=0, begin_time = 1593561600, queue_id = 420):
    
    container = []
    base_url = f"https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account"
    n_requests = 0
    
    for i in range(start, end):
        
        # reset begin index
        begin_index = 0
        
        # append a new list for each player
        # to keep count of number of entries.
        container.append([])
            
        while True:
            url= f"{base_url}/{players_list[i]}?queue={queue_id}&api_key={token}&beginTime={begin_time}&beginIndex={begin_index}"
                
            # control the number of requests
            if n_requests >= 100:
                print(f"Number of Entries so far: {len(container)}")
                n_requests = 0
                time.sleep(121)
                    
            # make request
            res = req.get(url)
            # update number of requests
            n_requests += 1
                
            # check for errors
            if res.status_code == 200:    
                # check if response is not empty
                if len(res.json()["matches"]) > 0:
                    container[i].append(res.json()["matches"])
                    # adjust begin index
                    begin_index += 100
                else:
                    break
            else:
                pd.DataFrame(container).to_pickle(f"raw_data/fragmented_data/match_ids_{i}.pkl", protocol=4)
                print(f"Something went wrong {res.status_code}")
                break

    # process container and save
    container = list(chain.from_iterable(list(chain.from_iterable(container))))
    container = pd.DataFrame(container)["gameId"]
    container.to_pickle("raw_data/match_ids.pkl")
    
    return f"File saved at raw_data/matches_id.pkl"

- Execute the function

In [27]:
cont = get_match_history(player_id_list, TOKEN, start=0, end=2)

- Check the file

In [4]:
matches = pd.read_pickle("raw_data/match_ids.pkl")
matches.head()

0    4545299712
1    4545284264
2    4545049730
3    4545120580
4    4544966065
Name: gameId, dtype: int64